In [1]:
!pip install wandb pytorch-lightning -qqq

In [3]:
import torch
import numpy as np
import glob
import matplotlib.pylab as plt
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchmetrics import Accuracy

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb

In [ ]:
# functions to show an image
def imshow(img,stats):
    img = img *stats[1] + stats[0]     # unnormalize
    npimg = img.numpy() # convert the tensor back to numpy
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Creating my own torch vision dataset
class TorchvisionDataset(Dataset):
    def __init__(self, data_dic, transform=None):
        self.file_paths = data_dic["X"]
        self.labels = data_dic["Y"]
        self.transform = transform
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        file_path = self.file_paths[idx]
        
        # Read an image with PIL
        image = Image.open(file_path)

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
print(torch.__version__)
# Check if GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

In [ ]:
# Listing the entire dataset
images = glob.glob(r"C:\\Users\\rober\\OneDrive - University of Calgary\\Documents\\Github\\deep-learning\\Data\\full_dataset2\\*\\*.jpg")
images = np.array(images)
labels = np.array([f.split("\\")[-2] for f in images])
unique_labels = np.unique(labels)
labels_integers = np.zeros(labels.shape, dtype = int)
for ii,jj in enumerate(unique_labels):
    labels_integers[labels == jj] = ii
print(labels_integers)

In [ ]:
batch_size = 32

# The first split gives you the test set 
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state = 0)
sss.get_n_splits(images, labels_integers)
dev_index, test_index = next(sss.split(images, labels_integers))
test_set = {"X": images[test_index], "Y": labels_integers[test_index] }


# The second split will give you the train and validation sets
dev_images = images[dev_index]
dev_labels = labels_integers[dev_index]
sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state = 0)
sss2.get_n_splits(dev_images, dev_labels)
train_index, val_index = next(sss2.split(dev_images, dev_labels))
train_set = {"X": dev_images[train_index], "Y": dev_labels[train_index] }
val_set = {"X": dev_images[val_index], "Y": dev_labels[val_index] }

print("Train set", train_set["X"].shape)
print("Val set",   val_set["X"].shape)
print("Test set",  test_set["X"].shape)

In [ ]:
# Mean and std were computed offline for the training set
torchvision_transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip()
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4120, 0.3768, 0.3407],
        std=[0.2944, 0.2759, 0.2598],
    )
])

torchvision_transform_test = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4120, 0.3768, 0.3407],
        std=[0.2944, 0.2759, 0.2598],
    )
])



train_dataset = TorchvisionDataset(train_set, transform= torchvision_transform)
val_dataset = TorchvisionDataset(val_set, transform= torchvision_transform)
test_dataset = TorchvisionDataset(test_set, transform= torchvision_transform_test)

# Get the data loader for the train set
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)



In [ ]:
#based on: https://colab.research.google.com/drive/1smfCw-quyKwxlj69bbsqpZhD75CnBuRh?usp=sharing#scrollTo=qu0xf25aUckF
class GarbageModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4, transfer=False):
        super().__init__()

         # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes
        
        # transfer learning if pretrained=True
        self.feature_extractor = models.resnet18(pretrained=transfer)

        if transfer:
            # layers are frozen by using eval()
            self.feature_extractor.eval()
            # freeze params
            for param in self.feature_extractor.parameters():
                param.requires_grad = False

            self.classifier = nn.Linear(self.feature_extractor.fc.in_features, num_classes)

            self.criterion = nn.CrossEntropyLoss()
            self.accuracy = Accuracy()
    
    # will be used during inference
    def forward(self, x):
       x = self.feature_extractor(x)
       x = x.view(x.size(0), -1)
       x = self.classifier(x)
       
       return x


In [ ]:
#wandb.login()

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 4)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)